# Dipole (bend) Example

In [ ]:
from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2

import impact.z as IZ

In [ ]:
reference_particle_mass = mec2
energy = 10e6
gamma = energy / mec2
sigma_x0 = 0.001
norm_emit_x = 1e-12
norm_emit_y = 1e-12
beta_x = sigma_x0**2 * gamma / norm_emit_x
beta_y = beta_x

n_particle = 1

In [ ]:
initial_particles = single_particle(pz=energy)

In [ ]:
input = IZ.ImpactZInput(
    # line 1
    ncpu_y=1,
    ncpu_z=1,
    # line 2
    seed=6,
    n_particle=n_particle,
    integrator_type="linear_map",
    err=1,
    diagnostic_type="extended",
    # line 3
    nx=32,
    ny=32,
    nz=32,
    boundary_type="trans_open_longi_open",
    radius_x=0.15,  # particles die after this point
    radius_y=0.15,
    z_period_size=0.0,  # TODO we think this is unused based on fortran code reading
    # line 4
    distribution="read",  # "gauss",
    restart=0,
    subcycle=0,  # TODO what is this?
    nbunch=1,
    # line 8
    twiss_alpha_x=0.0,
    twiss_beta_x=beta_x,
    twiss_norm_emit_x=norm_emit_x,
    twiss_mismatch_x=1.0,
    twiss_mismatch_px=1.0,
    twiss_offset_x=0.0,
    twiss_offset_px=0.0,
    # line 9
    twiss_alpha_y=0.0,
    twiss_beta_y=beta_y,
    twiss_norm_emit_y=norm_emit_y,
    twiss_mismatch_y=1.0,
    twiss_mismatch_py=1.0,
    twiss_offset_y=0.0,
    twiss_offset_py=0.0,
    # line 10
    twiss_alpha_z=0.0,
    twiss_beta_z=5e9,
    twiss_norm_emit_z=1e-9,
    twiss_mismatch_z=1.0,
    twiss_mismatch_e_z=1.0,
    twiss_offset_phase_z=0.0,
    twiss_offset_energy_z=0.0,
    # line 11
    # average_current=1.0,  # space charge on
    average_current=0.0,  # space charge off
    reference_kinetic_energy=energy - mec2,  # eV -> 10MeV total energy
    reference_particle_mass=reference_particle_mass,
    reference_particle_charge=-1.0,
    reference_frequency=1e9,  # arbitrarily set to 1GHz here
    initial_phase_ref=0.0,
    lattice=[
        IZ.WriteFull(file_id=1999, name="initial_particles"),
        # IZ.Drift(length=1e-9, steps=10, map_steps=10),  # <- to get initial particles, set this and set average_current to 0.0
        # IZ.Drift(length=1.0, radius=1.0, steps=10, map_steps=10, name="drift1"),
        IZ.Dipole(
            length=1.0,  # [m]
            steps=10,  # Number of space-charge kicks through the beamline element. Each "step" consists of a half-step, a space-charge kick, and another half-step.
            map_steps=10,  # Each half-step involves computing a map for that half-element which is computed by numerical integration.
            angle=1e-12,
            k1=0.0,
            input_switch=0.0,  # input switch 150., if >200, include 1D CSR
            hgap=0.0,
            e1=0.0,
            e2=0.0,
            entrance_curvature=0.0,
            exit_curvature=0.0,
            fint=0.0,
            misalignment_error_x=0.0,
            misalignment_error_y=0.0,
            rotation_error_x=0.0,
            rotation_error_y=0.0,
            rotation_error_z=0.0,
            name="dipole1",
        ),
        IZ.WriteFull(file_id=2000, name="final_particles"),
    ],
)

In [ ]:
input.space_charge_off()

In [ ]:
initial_particles.weight

In [ ]:
I = IZ.ImpactZ(input, initial_particles=initial_particles)

output = I.run(verbose=True)

In [ ]:
I.plot()

In [ ]:
I.output

In [ ]:
list(I.output.particles_raw)

In [ ]:
Pin = I.output.particles["initial_particles"]
# Our initial particles must be the same as what Impact-Z wrote:
assert initial_particles == Pin

In [ ]:
P_raw = I.output.particles_raw["final_particles"]
P = I.output.particles["final_particles"]

In [ ]:
I.plot("mean_x")